# TF-IDF based Recommender System

### Recommender System based on tf-idf as vector representation of documents

# TF-IDF Based Recommender
1. Represent articles in terms of bag of words
2. Represent user in terms of read articles associated words
3. Generate TF-IDF matrix for user read articles and unread articles
4. Calculate cosine similarity between user read articles and unread articles 
5. Get the recommended articles 

**Describing parameters**:

*1. PATH_NEWS_ARTICLES: specify the path where news_article.csv is present*  <br/>
*2. ARTICLES_READ: List of Article_Ids read by the user*  <br/>
*3. NO_RECOMMENDED_ARTICLES: Refers to the number of recommended articles as a result*

In [15]:
PATH_NEWS_ARTICLES="C:/Users/Arpan/Desktop/DS_Master/Reco/news_articles.csv"
ARTICLES_READ=[5,7]
NUM_RECOMMENDED_ARTICLES=5

In [2]:
try:
    import numpy
    import pandas as pd
    import pickle as pk
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity
    import re
    from nltk.stem.snowball import SnowballStemmer
    import nltk
    stemmer = SnowballStemmer("english")
except ImportError:
    print('You are missing some packages! ' \
          'We will try installing them before continuing!')
    !pip install "numpy" "pandas" "sklearn" "nltk"
    import numpy
    import pandas as pd
    import pickle as pk
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity
    import re
    from nltk.stem.snowball import SnowballStemmer
    import nltk
    stemmer = SnowballStemmer("english")
    print('Done!')

## 1. Represent articles in terms of bag of words

1. Reading the csv file to get the Article id, Title and News Content
2. Remove punctuation marks and other symbols from each article
3. Tokenize each article
4. Stem token of every article

In [3]:
news_articles = pd.read_csv(PATH_NEWS_ARTICLES)
news_articles.head()

,Article_Id,Title,Author,Date,Content,URL
0,0,14 dead after bus falls into canal in Telangan...,Devyani Sultania,"August 22, 2016 12:34 IST",At least 14 people died and 17 others were inj...,http://www.ibtimes.co.in/14-dead-after-bus-fal...
1,1,Pratibha Tiwari molested on busy road Saath ...,Suparno Sarkar,"August 22, 2016 19:47 IST",TV actress Pratibha Tiwari who is best known ...,NaN
2,2,US South Korea begin joint military drill ami...,Namrata Tripathi,"August 22, 2016 18:10 IST",The United States and South Korea began a join...,http://www.ibtimes.co.in/us-south-korea-begin-...
3,3,Illegal construction in Bengaluru Will my hou...,S V Krishnamachari,"August 22, 2016 17:39 IST",The relentless drive by Bengaluru s Bangalore...,http://www.ibtimes.co.in/illegal-construction-...
4,4,Punjab Gau Rakshak Dal chief held for assaulti...,Pranshu Rathee,"August 22, 2016 17:34 IST",Punjab Gau Raksha Dal chief Satish Kumar and h...,http://www.ibtimes.co.in/punjab-gau-rakshak-da...


In [5]:
#Select relevant columns and remove rows with missing values
news_articles = news_articles[['Article_Id','Title','Content']].dropna()
#articles is a list of all articles
articles = news_articles['Content'].tolist()
articles[5] #an uncleaned article

'Philippines police on Monday said that the number of drug-related deaths have doubled to about 1 800 since President Rodrigo Duterte assumed office in June  even as he threatened on Sunday to pull out of the United Nations  UN  and invite China and others to form a new organisation \nHe made these comments in response to two United Nations human rights experts  slamming the newly formed government for encouraging extra-judicial killings to eradicate the drug menace in the Philippines \n\n\nDuterte s comments\n I will prove to the world that you are a very stupid expert  I do not want to insult you  But maybe we ll just have to decide to separate from the United Nations  Why do you have to listen to this stupid   said President Duterte  in a middle-of-the-night news conference  in his home town  Davao  as he urged his critics to count not just the number of drug-related deaths  but also the innocent lives lost to drugs \nHe continued criticizing the United Nations saying it was not abl

In [6]:
def clean_tokenize(document):
    document = re.sub('[^\w_\s-]', ' ',document)       #remove punctuation marks and other symbols
    tokens = nltk.word_tokenize(document)              #Tokenize sentences
    cleaned_article = ' '.join([stemmer.stem(item) for item in tokens])    #Stemming each token
    return cleaned_article

In [8]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


KeyboardInterrupt: 

In [9]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('maxent_ne_chunker')
nltk.download('averaged_perceptron_tagger')
nltk.download('words')   

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arpan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arpan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\arpan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping chunkers\maxent_ne_chunker.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\arpan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\arpan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.


True

In [10]:
cleaned_articles = map(clean_tokenize, articles)
cleaned_articles[0]  #a cleaned, tokenized and stemmed article 

u'at least 14 peopl die and 17 other were injur after a bus travel from hyderabad to kakinada plung into a canal from a bridg on the accident-pron stretch of the hyderabad-khammam highway in telangana earli monday morn the injur were admit to the govern general hospit for treatment seven peopl die on the spot and the other succumb to injuri while undergo treatment at the hospit the passeng belong to the east and west godavari district of andhra pradesh the bus own by privat oper yatra geni commenc it journey from hyderabad at 11 30 p m on sunday khammam superintend of polic shah nawaz khan was quot by the hindustan time as say the accid happen around 2 30 a m when the driver slam the brake to avoid a collis with anoth vehicl come from the opposit direct on a bridg over nagarjunsagar project left canal at nayankangudem villag in khammam district the daili report the bus hit the parapet wall of the bridg and nose-div into the canal the driver of the bus was appar drive at high speed due 

# 2. Represent user in terms of read articles associated words


In [17]:
#Get user representation in terms of words associated with read articles
user_articles = ' '.join(cleaned_articles[i] for i in ARTICLES_READ)

In [18]:
user_articles

u'philippin polic on monday said that the number of drug-rel death have doubl to about 1 800 sinc presid rodrigo dutert assum offic in june even as he threaten on sunday to pull out of the unit nation un and invit china and other to form a new organis he made these comment in respons to two unit nation human right expert slam the newli form govern for encourag extra-judici kill to erad the drug menac in the philippin dutert s comment i will prove to the world that you are a veri stupid expert i do not want to insult you but mayb we ll just have to decid to separ from the unit nation whi do you have to listen to this stupid said presid dutert in a middle-of-the-night news confer in his home town davao as he urg his critic to count not just the number of drug-rel death but also the innoc live lost to drug he continu critic the unit nation say it was not abl to fulfil it own mandat but was instead worri about the bone of crimin pile up 6 7 prime minist narendra modi has express deep conce

# 3. Generate TF-IDF matrix for user read articles and unread articles


In [19]:
#Generate tfidf matrix model for entire corpus
tfidf_matrix = TfidfVectorizer(stop_words='english', min_df=2)
article_tfidf_matrix = tfidf_matrix.fit_transform(cleaned_articles)
article_tfidf_matrix #tfidf vector of an article

<4831x16009 sparse matrix of type '<type 'numpy.float64'>'
	with 468648 stored elements in Compressed Sparse Row format>

In [20]:
#Generate tfidf matrix model for read articles
user_article_tfidf_vector = tfidf_matrix.transform([user_articles])
user_article_tfidf_vector

<1x16009 sparse matrix of type '<type 'numpy.float64'>'
	with 172 stored elements in Compressed Sparse Row format>

In [21]:
user_article_tfidf_vector.toarray()

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.]])

# 4. Calculate cosine similarity between user read articles and unread articles 



In [22]:
articles_similarity_score=cosine_similarity(article_tfidf_matrix, user_article_tfidf_vector)

In [23]:
recommended_articles_id = articles_similarity_score.flatten().argsort()[::-1]

In [24]:
recommended_articles_id

array([   7,    5, 2862, ...,  595,  233,  693], dtype=int64)

In [25]:
#Remove read articles from recommendations
final_recommended_articles_id = [article_id for article_id in recommended_articles_id 
                                 if article_id not in ARTICLES_READ ][:NUM_RECOMMENDED_ARTICLES]

# 5. Get the recommended articles 

In [26]:
final_recommended_articles_id

[2862, 2808, 2703, 2724, 2936]

In [27]:
#Recommended Articles and their title
print 'Articles Read'
print news_articles.loc[news_articles['Article_Id'].isin(ARTICLES_READ)]['Title']
print '\n'
print 'Recommender '
print news_articles.loc[news_articles['Article_Id'].isin(final_recommended_articles_id)]['Title']

Articles Read
5    Phillipines drug war  1 800 drug-related death...
7    Dialogue crucial in finding permanent solution...
Name: Title, dtype: object


Recommender 
2703    Home Min Rajnath Singh hints at stopping use o...
2724    PM Modi says at all-party meeting that PoK is ...
2808    J K  CM Mufti blames  vested interests  for Ka...
2862    J K  PM Modi appeals for peace in Valley  assu...
2936    Home Minister Rajnath Singh in Kashmir  To mee...
Name: Title, dtype: object


In [28]:
articles[0]

'At least 14 people died and 17 others were injured after a bus travelling from Hyderabad to Kakinada plunged into a canal from a bridge on the accident-prone stretch of the Hyderabad-Khammam highway in Telangana early Monday morning \nThe injured were admitted to the Government General Hospital for treatment \n\n\nSeven people died on the spot and the others succumbed to injuries while undergoing treatment at the hospital  The passengers belonged to the East and West Godavari districts of Andhra Pradesh \nThe bus  owned by private operator Yatra Genie  commenced its journey from Hyderabad at 11 30 p m  on Sunday  Khammam Superintendent of Police Shah Nawaz Khan was quoted by the Hindustan Times as saying \nThe accident happened around 2 30 a m  when the driver slammed the brakes to avoid a collision with another vehicle coming from the opposite direction on a bridge over Nagarjunsagar project left canal at Nayankangudem village in Khammam district  the daily reported  The bus hit the 

In [29]:
cleaned_articles[0]

u'at least 14 peopl die and 17 other were injur after a bus travel from hyderabad to kakinada plung into a canal from a bridg on the accident-pron stretch of the hyderabad-khammam highway in telangana earli monday morn the injur were admit to the govern general hospit for treatment seven peopl die on the spot and the other succumb to injuri while undergo treatment at the hospit the passeng belong to the east and west godavari district of andhra pradesh the bus own by privat oper yatra geni commenc it journey from hyderabad at 11 30 p m on sunday khammam superintend of polic shah nawaz khan was quot by the hindustan time as say the accid happen around 2 30 a m when the driver slam the brake to avoid a collis with anoth vehicl come from the opposit direct on a bridg over nagarjunsagar project left canal at nayankangudem villag in khammam district the daili report the bus hit the parapet wall of the bridg and nose-div into the canal the driver of the bus was appar drive at high speed due 